<a href="https://colab.research.google.com/github/cmd-gunma-univ/DX-highschool/blob/main/DHT_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pinout

＋  白   3.3V     (1)    
out 黒   GPIO14 (8)   
➖  茶色 GND     (9)

In [ ]:
# config.txtに追記
!echo "dtoverlay=dht11,gpiopin=14" | sudo tee -a /boot/firmware/config.txt

dtoverlay=dht11,gpiopin=14


In [ ]:
# 追記を確認
!tail -n 5 /boot/firmware/config.txt

[cm5]
dtoverlay=dwc2,dr_mode=host

[all]
dtoverlay=dht11,gpiopin=14


In [ ]:
# 再起動
! sudo reboot

In [ ]:
# 2秒に一回読み取るコマンド
import time

device0 = "/sys/bus/iio/devices/iio:device0"

# ファイルの先頭行を読み取って整数値を返す関数
def readFirstLine(filename):
    try:
        f = open(filename,"rt")
        value = int(f.readline())
        f.close()
        return True, value
    except ValueError:
        f.close()
        return False, -1
    except OSError:
        return False, 0

try:
    while True:
        # 温度を取得
        Flag, Temperature = readFirstLine(device0 + "/in_temp_input")
        print("Temperature:", end="")
        if Flag:
            print(Temperature // 1000, "℃", end="\t")
        else:
            print("N.A.", end="\t")

        # 湿度を取得
        Flag, Humidity = readFirstLine(device0 + "/in_humidityrelative_input")
        print("Humidity:", end="")
        if Flag:
            print(Humidity // 1000, "%")
        else:
            print("N.A.")

        time.sleep(2.0)
except KeyboardInterrupt:
    pass



In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from collections import deque

device0 = "/sys/bus/iio/devices/iio:device0"

# --- 値読み取り関数 ---
def readFirstLine(filename):
    try:
        with open(filename, "rt") as f:
            value = int(f.readline())
        return True, value
    except (ValueError, OSError):
        return False, 0

# --- データ保持 ---
max_points = 60  # 最新60点を表示
times = deque(maxlen=max_points)
temps = deque(maxlen=max_points)
hums = deque(maxlen=max_points)

start_time = time.time()

# --- グラフ設定 ---
plt.rcParams["figure.figsize"] = (8, 4)

try:
    for i in range(100):  # ループ回数を制限（例：100回 ≒ 3分程度）
        now = time.time() - start_time

        # 温度
        Flag, Temperature = readFirstLine(device0 + "/in_temp_input")
        Temperature = Temperature / 1000.0 if Flag else None

        # 湿度
        Flag, Humidity = readFirstLine(device0 + "/in_humidityrelative_input")
        Humidity = Humidity / 1000.0 if Flag else None

        times.append(now)
        temps.append(Temperature)
        hums.append(Humidity)

        # --- グラフ描画 ---
        clear_output(wait=True)
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()

        ax1.plot(times, temps, color='tab:red', label='Temperature (°C)')
        ax2.plot(times, hums, color='tab:blue', label='Humidity (%)')

        ax1.set_xlabel("Time [s]")
        ax1.set_ylabel("Temperature [°C]", color='tab:red')
        ax2.set_ylabel("Humidity [%]", color='tab:blue')
        ax1.set_title("Real-time Temperature & Humidity")

        plt.show()

        time.sleep(2.0)  # 2秒ごとに更新

except KeyboardInterrupt:
    print("⏹ 中断しました。")